# Two-qubit complex wavefunction

In this tutorial, we use QuCumber reconstruct the most likely wavefunction from measurements of tow qubits. In constrast with the TFIM tutorial, this wavefunction as an amplitude and a phase, so it cannot be written as a completely positive real wavefunction in any basis.

### Two qubits
The two qubits are 
\begin{equation}
            |\psi \rangle = \alpha |00\rangle + \beta | 01\rangle + \gamma |10\rangle + \delta 11\rangle
\end{equation}
where $\alpha, \beta, \gamma, \delta$ we want to approximate. The exact values used for this tutorial are in qubits_psi.txt, and are 
\begin{equation}
\alpha =0.2860859781  + 0.0538594435 i \\
\beta = 0.3686925645 - 0.3022891852 i \\
\gamma = -0.1672402652 - 0.3528898162 i \\
\delta = -0.5658788296 - 0.4639198598 i
\end{equation}

##### Code 
As before, we load the required Python packages

In [ ]:
import numpy as np
import pickle
import torch

from qucumber.nn_states import ComplexWavefunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber.utils.cplx as cplx
import qucumber.utils.unitaries as unitaries

## Training Data

The training data for a complex wavefunction consists of two file. Firstly, the basis which are measured must be listed. In our example, this is in "qubits_train_bases.txt" Secondly, we need the values of the measurements (0, 1) from measurement with those opterators. This is included in "ubits_train.txt"and we encourage the reader to take a look at both files to see the format. 

We also need a list of all the unique basis measurements from the "quits_train_bases.txt"file. This can be easily found with numpy.

In [ ]:
a = np.loadtxt("qubits_train_bases.txt", dtype=str)
bases = np.unique(a, axis=0)
bases = ["".join(bases[i, :]) for i in range(bases.shape[0])]
bases

The remainder of the training data can be loaded with the utility function qucumber.utils.data.load_data(). This function simply read the .txt files, and converts them to torch tensors. 

In [ ]:
train_path = "qubits_train.txt"
train_bases_path = "qubits_train_bases.txt"
psi_path = "qubits_psi.txt"
train, psi, train_bases = data.load_data(train_path, psi_path, train_bases_path)

To construct a **ComplexWavefunction** neural network state we need to create a dictionary that contains the unitaries that have been applied during the measurements. In our case this were the unitaries $\mathbb{1}$, $H$, $K$.

In [ ]:
unitary_dict = unitaries.create_dict()
"""If you would like to add your own quantum gates from your experiment to 
   "unitary_dict", do:
   unitary_dict = unitaries.create_dict(name='your_name', 
                                        unitary=torch.tensor([[real part], 
                                                              [imaginary part]], 
                                                             dtype=torch.double)
                                                             
   For example: 
   unitaries = unitary_library.create_dict(name='qucumber', 
                                           unitary=torch.tensor([ [[1.,0.],[0.,1.]] 
                                                                  [[0.,0.],[0.,0.]] ], 
                                                                dtype=torch.double))
                                                                                             
   By default, unitary_library.create_dict() contains the idenity matrix and the 
   hadamard and K gates with keys Z, X and Y, respectively.
"""

unitary_dict

The number of visible units is equal to the number of qubits or the length of a training sample. The number of hidden units is set equal to the number of visible units for a neuron density of $\alpha=1$.

Again GPU computing is supported by setting  "gpu = True" in ComplexWavefunction.

In [ ]:
nn_state = ComplexWavefunction(num_visible=2, num_hidden=2, unitary_dict=unitary_dict, gpu=False)

To evaluate how the RBM is training, we will compute the full KL divergence and the fidelity between the true wavefunction of the system and the wavefunction the RBM reconstructs.
This can be done by initializing the parameters of the **ComplexWavefunction** and the **MetricEvaluator**.

In [ ]:
log_every = 10

nn_state.space = nn_state.generate_hilbert_space(2)
callbacks = [
    MetricEvaluator(
        log_every,
        {"Fidelity": ts.fidelity, "KL": ts.KL},
        target_psi=psi,
        bases=bases,
        verbose=True,
        space=nn_state.space,
    )
]

After initializing everything we can start the training.

In [ ]:
nn_state.fit(
    train,
    epochs=100,
    pos_batch_size=50,
    neg_batch_size=10,
    lr=1e-2,
    k=2,
    input_bases=train_bases,
    progbar=True,
    callbacks=callbacks,
)

### After Training 

After the training we can calculate state fidelity, observables or sample from the complex wavefunction
the same way we did from the real-positive wavefunction. However, one has to keep in mind that the sampling only works in the Z basis.

To sample from a trained complex wavefunction we define the number of samples *num_samples* we want to draw and the number of contrastive divergence steps *CD*. 

In [ ]:
num_samples = 2000
CD = 200

samples = nn_state.sample(num_samples, CD)

Also analogous to the positive real wavefunction we can save and load the RBM parameters and the newly generated samples using the *save* function within the ComplexWavefunction object and save additional quantities like e.g. *the samples* to the same file with *metadata*.

In [ ]:
nn_state.save("saved_parameters.pkl", metadata={"Samples": samples})